In [0]:
%matplotlib inline

## Hướng dẫn sử dụng PyTorch
### PyTorch là gì?
Pytorch là thư viện deep learning có thể chạy trên GPU. Pytorch tương tự như Numpy và hoàn toàn có thể thay thế Numpy.

In [0]:
import numpy as np
import torch
# kiểm tra phiên bản của pytorch >= 1.0
print("torch version:", torch.__version__)

In [0]:
!ls

Cách khởi tạo một tensor trong Pytorch. Các hàm trong Pytorch tương tự như Numpy

In [0]:
print('\nCreate a zero ndarray in NumPy:')
zero_np = np.zeros([2, 3])
print(zero_np)
print('\nCreate a zero tensor in PyTorch:')
zero_pt = torch.zeros([2,3])
print(zero_pt)

Truy xuất phần tử của tensor theo index



In [0]:
print("numpy: zero_np[0,1]: {}\t(type: {})".format(str(zero_np[0,1]), type(zero_np[0,1])))
print("torch: zero_pt[0,1]: {}\t(type: {} / shape: {})".format(str(zero_pt[0,1]), type(zero_pt[0,1]), zero_pt[0,1].shape))
# Use "item()" to get a Python number from a single-valued tensor.
print("       zero_pt[0,1].item(): {}\t(type: {})".format(zero_pt[0,1].item(), type(zero_pt[0,1].item())))

In [0]:
print(zero_pt.size())
print(zero_pt.type())
print(zero_pt[0,1].item())

Chuyển đổi từ Numpy sang Pytorch và ngược lại

In [0]:
print('Turn a ndarray into a tensor with "torch.tensor()":')
zero_pt_from_np = torch.tensor(zero_np)
print(zero_pt_from_np)
print('or "torch.from_numpy():"')
zero_pt_from_np = torch.from_numpy(zero_np)
print(zero_pt_from_np)

print('\nTurn a tensor into ndarray with ".numpy()":')
zero_np_from_pt = zero_pt.numpy()
print(zero_np_from_pt)
print(type(zero_np_from_pt))

In [0]:
a = torch.from_numpy(np.zeros([1,2]))
b = torch.zeros([1,2])
b+a

Chuyển một tensor lên GPU và ngược lại

In [0]:
!nvidia-smi

In [0]:
t = torch.randn(2)
print("Initial device:\t'{}'".format(t.device))
t = t.to('cuda:0')
print("Move to gpu:\t'{}'".format(t.device))
t = t.to('cpu')
print("Back to cpu:\t'{}'".format(t.device))
t.numpy()

Cơ chế Autograd của Pytorch

In [0]:
x = torch.randn(5, 5)  # requires_grad=False by default
y = torch.randn(5, 5)  # requires_grad=False by default
z = torch.randn((5, 5), requires_grad=True)
a = x + y
print(a.requires_grad)
b = a + z
print(b.requires_grad)

Tính đạo hàm $\frac{dc}{dx}$

In [0]:
x = torch.tensor([1.0])
y = torch.tensor([2.0])
z = torch.tensor([3.0])
x.requires_grad_()
c = x * y + z
print(c)
print(x.grad)
c.backward()
print(x)
print(x.grad)

Gán `requires_grad=False` có tác dụng khi dùng một mô hình pretrain (ví dụ VGG, Resnet, ...) có sẵn làm mạng nền cho một mô hình khác (ví dụ mô hình mạng phát hiện SSD). Khi đó mạng nền sẽ được "đóng băng" và sẽ không được cập nhật trọng số, trong khi đó các phần còn lại của mạng sẽ được cập nhật trọng số bình thường

In [0]:
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18

In [0]:
model = resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
# Replace the last fully-connected layer
# Parameters of newly constructed modules have requires_grad=True by default
model.fc = nn.Linear(512, 100)

# Optimize only the classifier
optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)

Huấn luyện mạng phân loại MNIST
=====
## Các bước để huấn luyện một mạng:
 - Chuẩn bị dữ liệu
 - Tạo mạng
 - Chọn bộ optimizer
 - Huấn luyện mô hình
 - Đánh giá kết quả

In [0]:
import torch.nn as nn            # containing various building blocks for your neural networks
import torch.optim as optim      # implementing various optimization algorithms
import torch.nn.functional as F  # a lower level (compared to torch.nn) interface

# torchvision: popular datasets, model architectures, and common image transformations for computer vision.
import torchvision
# transforms: transformations useful for image processing
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader

import glob
import os.path as osp
import numpy as np
import cv2

## 1. Chuẩn bị dữ liệu
Chạy các câu lệnh sau để tải tập mnist:

`wget https://github.com/myleott/mnist_png/blob/master/mnist_png.tar.gz?raw=true -O mnist_png.tar.gz`

`tar -xzf mnist_png.tar.gz`

In [0]:
!wget https://github.com/myleott/mnist_png/blob/master/mnist_png.tar.gz?raw=true -O mnist_png.tar.gz
!tar -xzf mnist_png.tar.gz

Trong gói torchvision có hiện thực sẵn một vài [dataset](https://pytorch.org/docs/stable/torchvision/datasets.html)

In [0]:
trainset = torchvision.datasets.MNIST('/content', train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
testset = torchvision.datasets.MNIST('/content', train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

In [0]:
trainset = torchvision.datasets.CIFAR10('/content', train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
testset = torchvision.datasets.CIFAR10('/content', train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

In [0]:
trainset = torchvision.datasets.CIFAR100('/content', train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
testset = torchvision.datasets.CIFAR100('/content', train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

In [0]:
class MNIST(Dataset):
    """
    A customized data loader for MNIST.
    """
    def __init__(self,
                 root,
                 transform=None,
                 preload=False):
        """ Intialize the MNIST dataset
        
        Args:
            - root: root directory of the dataset
            - tranform: a custom tranform function
            - preload: if preload the dataset into memory
        """
        self.images = None
        self.labels = None
        self.filenames = []
        self.root = root
        self.transform = transform

        # read filenames
        for i in range(10):
            filenames = glob.glob(osp.join(root, str(i), '*.png'))
            for fn in filenames:
                self.filenames.append((fn, i)) # (filename, label) pair
                
        # if preload dataset into memory
        if preload:
            self._preload()
            
        self.len = len(self.filenames)
                              
    def _preload(self):
        """
        Preload dataset to memory
        """
        self.labels = []
        self.images = []
        for image_fn, label in self.filenames:            
            # load images
            image = cv2.imread(image_fn)
            self.images.append(image.copy())
            self.labels.append(label)

    # probably the most important to customize.
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        if self.images is not None:
            # If dataset is preloaded
            image = self.images[index]
            label = self.labels[index]
        else:
            # If on-demand data loading
            image_fn, label = self.filenames[index]
            image = cv2.imread(image_fn)
            
        # May use transform function to transform samples
        # e.g., random crop, whitening
        if self.transform is not None:
            image = self.transform(image)
        # return image and label
        return image, label

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

Sử dụng dataset tự tạo

In [0]:
# Create the MNIST dataset. 
# transforms.ToTensor() automatically converts PIL images to
# torch tensors with range [0, 1]
trainset = MNIST(
    root='mnist_png/training',
    preload=True, transform=transforms.ToTensor(),
)

# Load the testset
testset = MNIST(
    root='mnist_png/testing',
    preload=True, transform=transforms.ToTensor(),
)

Dữ liệu trước khi được đưa vào huấn luyện có thể trải qua một vài phép biến đổi. Đó được gọi là làm giàu dữ liệu. Một số hàm biến đổi tham khảo [link](https://pytorch.org/docs/stable/torchvision/transforms.html#transforms-on-pil-image)

In [0]:
# Use the torch dataloader to iterate through the dataset
# We want the dataset to be shuffled during training.
trainset_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=1)

# Use the torch dataloader to iterate through the dataset
testset_loader = DataLoader(testset, batch_size=1000, shuffle=False, num_workers=1)

In [0]:
print(len(trainset)) # len = 60000
print(len(testset))  # len = 10000

### Hiển thị dataset

In [0]:
import matplotlib.pyplot as plt

# functions to show an image
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

# get some random training images
dataiter = iter(trainset_loader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % labels[j] for j in range(16)))

In [0]:
# Use GPU if available, otherwise stick with cpu
use_cuda = torch.cuda.is_available()
torch.manual_seed(123)
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

## Tạo mạng

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0,
        #        dilation=1, groups=1, bias=True, padding_mode='zeros')
        # NCHW
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        
        # Linear(in_features, out_features, bias=True)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
        # MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
        self.max_pool = nn.MaxPool2d(2)
        # ReLU(inplace=False)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Note: the following two ways for max pooling / relu are equivalent.
        # 1) with torch.nn.functional:
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        # 2) with torch.nn:
        x = self.relu(self.max_pool(self.conv2_drop(self.conv2(x))))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

### Huấn luyện mạng

In [0]:
from time import time
def train(epoch, log_interval=100):
    model.train()  # set training mode
    iteration = 0
    for ep in range(epoch):
        start = time()
        for batch_idx, (data, target) in enumerate(trainset_loader):
            # bring data to the computing device, e.g. GPU
            data, target = data.to(device), target.to(device)

            # forward pass
            output = model(data)
            # compute loss: negative log-likelihood
            loss = F.nll_loss(output, target)
            
            # backward pass
            # clear the gradients of all tensors being optimized.
            optimizer.zero_grad()
            # accumulate (i.e. add) the gradients from this forward pass
            loss.backward()
            # performs a single optimization step (parameter update)
            optimizer.step()
            
            if iteration % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    ep, batch_idx * len(data), len(trainset_loader.dataset),
                    100. * batch_idx / len(trainset_loader), loss.item()))
            iteration += 1
            
        end = time()
        print('{:.2f}s'.format(end-start))
        test() # evaluate at the end of epoch

In [0]:
def test():
    model.eval()  # set evaluation mode
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in testset_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(testset_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(testset_loader.dataset),
        100. * correct / len(testset_loader.dataset)))

In [0]:
train(5)  # train 5 epochs should get you to about 97% accuracy

### Lưu mô hình đã train

In [0]:
def save_checkpoint(checkpoint_path, model, optimizer):
    # state_dict: a Python dictionary object that:
    # - for a model, maps each layer to its parameter tensor;
    # - for an optimizer, contains info about the optimizer’s states and hyperparameters used.
    state = {
        'state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict()}
    torch.save(state, checkpoint_path)
    print('model saved to %s' % checkpoint_path)
    
def load_checkpoint(checkpoint_path, model, optimizer):
    state = torch.load(checkpoint_path)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    print('model loaded from %s' % checkpoint_path)

In [0]:
# create a brand new model
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# Testing -- you should get a pretty poor performance since the model hasn't learned anything yet.
test()

Lưu mô hình

In [0]:
def train_save(epoch, save_interval, log_interval=100):
    model.train()  # set training mode
    iteration = 0
    for ep in range(epoch):
        for batch_idx, (data, target) in enumerate(trainset_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            if iteration % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    ep, batch_idx * len(data), len(trainset_loader.dataset),
                    100. * batch_idx / len(trainset_loader), loss.item()))
            # different from before: saving model checkpoints
            if iteration % save_interval == 0 and iteration > 0:
                save_checkpoint('mnist-%i.pth' % iteration, model, optimizer)
            iteration += 1
        test()
    
    # save the final model
    save_checkpoint('mnist-%i.pth' % iteration, model, optimizer)

In [0]:
train_save(5, save_interval=500, log_interval=100)

In [0]:
a = torch.load('/content/mnist-1000.pth')

In [0]:
print(a.keys())

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# create a new model
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# load from the final checkpoint
load_checkpoint('mnist-4690.pth', model, optimizer)
# should give you the final model accuracy
test()

# email: nqtran.sdh19@hcmut.edu.vn

### Fine-tune mô hình

In [0]:
# What's in a state dict?
print(model.state_dict().keys())

In [0]:
checkpoint = torch.load('mnist-4690.pth')
states_to_load = {}
for name, param in checkpoint['state_dict'].items():
    if name.startswith('conv'):
        # only load the conv layers
        states_to_load[name] = param
print("Number of parameter variables to load:", len(states_to_load))

# Construct a new state_dict in which the layers we want
# to import from the checkpoint is updated with the parameters
# from the checkpoint
model = Net().to(device)
model_state = model.state_dict()
print("Number of parameter variables in the model:", len(model_state))
model_state.update(states_to_load)
        
model.load_state_dict(model_state)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [0]:
test() # without fine-tuning.

train(1)  # training 1 epoch will get you to 93%!
# As a comparison, training from scratch for 1 epoch gets about ~80% test accuracy.

Sử dụng mô hình pretrained

In [0]:
class SmallNet(nn.Module):
    def __init__(self):
        super(SmallNet, self).__init__()
        # same conv layers
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        # fewer FC layers
        self.fc1 = nn.Linear(320, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)

model = SmallNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [0]:
checkpoint = torch.load('mnist-4690.pth')
states_to_load = {}
for name, param in checkpoint['state_dict'].items():
    if name.startswith('conv'):
        states_to_load[name] = param

# Construct a new state dict in which the layers we want
# to import from the checkpoint is update with the parameters
# from the checkpoint
model_state = model.state_dict()
model_state.update(states_to_load)
        
test()

model.load_state_dict(model_state)

In [0]:
train(1)  # training 1 epoch will get you to ~93%!

### Sử dụng nn.Sequential

In [0]:
class NetSeq(nn.Module):
    def __init__(self):
        super(NetSeq, self).__init__()

        # conv layers: feature extractor
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout2d(),
            nn.MaxPool2d(2),
            nn.ReLU()
        )
        
        # fc layers: classifier
        self.fc_layers = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(50, 10),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 320)
        x = self.fc_layers(x)
        return F.log_softmax(x, dim=1)

model = NetSeq().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [0]:
train(5)